# Repository

In [ ]:
try:
  from google.colab import drive
  drive.mount('/gdrive')
except:
  print("vous n'êtes encore  sur Google Colab")

Mounted at /gdrive


In [ ]:
%cd '/gdrive/My Drive/Colab Notebooks/Dedoublement/'

/gdrive/My Drive/Colab Notebooks/Dedoublement


In [ ]:
import os  
REP_PROJET = os.getcwd()
REP_INPUT  = REP_PROJET + '/Input/'
REP_INTERMED  = REP_PROJET + '/Intermed/'
REP_OUTPUT  = REP_PROJET + '/Output/'

# Requirements

In [ ]:
import sys
import subprocess
import pkg_resources

requirements = set(open('requirements.txt').read().split())
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = requirements - installed
if missing:
    subprocess.check_call([sys.executable, '-m','pip', 'install', *missing], stdout=subprocess.DEVNULL)

# Modules

In [ ]:
from pathlib import Path
import pandas as pd
from functions.functions_knn import *
from functions.vectorization import *

# Inputs

In [ ]:
baseline = pd.read_pickle(REP_INTERMED + 'baseline.sav')
cdandlp =  pd.read_pickle(REP_INTERMED + 'base_cdandlp.sav')
cdandlp['text']  = cdandlp['artiste']  + ' ' +  cdandlp['titre']

analyzer = 'char'
ngram_range = (2,2)
X_train, vectorizer = reload_data_and_vectorizer(analyzer = analyzer, ngram_range=ngram_range)

with open(REP_INTERMED + 'dict_words.pkl', 'rb') as f:
        dict_words = pickle.load(f) 

In [ ]:
taille = len(cdandlp)
pas = int(taille/37)
list_tmp = [ indices for indices in range(0,  taille+1, pas) ]


In [ ]:
[ 
 level2_knn_words(start, 
                   end,
                   baseline = baseline, 
                   dict_words=dict_words,
                   df_cdandlp =cdandlp,
                   X_train = X_train, 
                   vectorizer =vectorizer
                  ) 
  for start, end in zip(list_tmp,list_tmp[1:])
                   
]

In [ ]:
## Concaténation et lecture des csv

dfs = [pd.concat(pd.read_pickle(path), ignore_index=True) for path in list(Path(REP_INTERMED).glob('*All_words_knn*'))]

df_final = pd.concat(dfs, ignore_index=True)
df_final.to_csv(REP_OUTPUT + "resultats_knn_All.csv", index=False, encoding='utf-8-sig')
df_final.to_pickle(REP_OUTPUT + "resultats_knn_All.pkl")

In [ ]:
%reset-f 